In [39]:
import requests
from bs4 import BeautifulSoup
import re

import pandas as pd

def recuperer_contenu(url, nom_classe):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Erreur lors de la requête : {e}")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    balises = soup.find_all(class_=nom_classe)
    
    return balises

In [40]:
# Recuperation des url 

import json
def read_json_file(file_path):
    try:
        # Ouverture et lecture du fichier JSON
        with open(file_path, 'r', encoding='utf-8') as file:
            # Conversion du contenu JSON en dictionnaire Python
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Erreur: Le fichier {file_path} n'existe pas")
        return None
    except json.JSONDecodeError:
        print(f"Erreur: Le fichier {file_path} n'est pas un JSON valide")
        return None
    except Exception as e:
        print(f"Une erreur inattendue s'est produite: {str(e)}")
        return None

def get_ship(data, name):
    return data[name]
    

def get_url(data,name):
    url = get_ship(data,name)['pledge_url']
    return url


data = read_json_file('shipDB_All.json')

# creation d une liste contenant toutes les url 
 
def list_ship(data):
    Flight_ready=[]
    In_Concept = []
    for i in data:
        if 'pledge_url' in data[i].keys():
            Flight_ready.append(i)
        else : 
            In_Concept.append(i)
    return Flight_ready,In_Concept 
    

In [42]:
Flight_ready = [get_ship(data,i)['pledge_url']  for i in list_ship(data)[0]] 

Flight_ready

['https://robertsspaceindustries.com/pledge/ships/origin-100/100i',
 'https://robertsspaceindustries.com/pledge/ships/origin-100/125a',
 'https://robertsspaceindustries.com/pledge/ships/origin-100/135c',
 'https://robertsspaceindustries.com/pledge/ships/origin-300/300i',
 'https://robertsspaceindustries.com/pledge/ships/origin-300/315p',
 'https://robertsspaceindustries.com/pledge/ships/origin-300/325a',
 'https://robertsspaceindustries.com/pledge/ships/origin-300/350r',
 'https://robertsspaceindustries.com/pledge/ships/400i/400i',
 'https://robertsspaceindustries.com/pledge/ships/600i/600i-Explorer',
 'https://robertsspaceindustries.com/pledge/ships/600i/600i-Touring',
 'https://robertsspaceindustries.com/pledge/ships/85x/85X',
 'https://robertsspaceindustries.com/pledge/ships/890-jump/890-Jump',
 'https://robertsspaceindustries.com/pledge/ships/spirit/A1-Spirit',
 'https://robertsspaceindustries.com/pledge/ships/crusader-starlifter/A2-Hercules',
 'https://robertsspaceindustries.com/p

In [86]:
#creation du dataframe

def DataBase():
    nomclasse = "l-sheet__block"
    dict = {}
    liste_dict=[]
    for i in Flight_ready[:]:
        dict = {}
        for e in recuperer_contenu(i, nomclasse):
            e = e.text.replace("\n", "|").replace("\t", "")
            e = e.split("|")
            n = []
            for i in e:
                i = re.sub(r'\s+', ' ', i).strip()
                if len(i)!=0:
                    n.append(i)
            for i in range(0, len(n)-2, 2):
                dict[n[i]] = n[i+1]
        print(dict)
        #dict=fusioncomplete(dict,dict)
        liste_dict.append(dict)
    dict=fusion_complete( liste_dict)

    return dict

DB = DataBase()

{'Focus': 'Starter / Touring', 'Length': '19.3m', 'Beam': '11.0m', 'Height': '4.0m', 'Size': 'Small', 'Cargo Capacity': '2', 'SCM Speed': '260 m/s', 'Afterburner Speed': '1,425 m/s', 'Min Crew': '1', 'Pitch Max': '- deg/s', 'Yaw Max': '- deg/s', 'Roll Max': '- deg/s', 'X-Axis Acceleration': '- m/s/s', 'Y-Axis Acceleration': '- m/s/s'}
{'Focus': 'Starter / Light Fighter', 'Length': '19.3m', 'Beam': '11.0m', 'Height': '4.0m', 'Size': 'Small', 'Cargo Capacity': '2', 'SCM Speed': '260 m/s', 'Afterburner Speed': '1,425 m/s', 'Min Crew': '1', 'Pitch Max': '- deg/s', 'Yaw Max': '- deg/s', 'Roll Max': '- deg/s', 'X-Axis Acceleration': '- m/s/s', 'Y-Axis Acceleration': '- m/s/s'}
{'Focus': 'Starter / Light Freight', 'Length': '19.3m', 'Beam': '11.0m', 'Height': '4.0m', 'Size': 'Small', 'Cargo Capacity': '6', 'SCM Speed': '260 m/s', 'Afterburner Speed': '1,425 m/s', 'Min Crew': '1', 'Pitch Max': '- deg/s', 'Yaw Max': '- deg/s', 'Roll Max': '- deg/s', 'X-Axis Acceleration': '- m/s/s', 'Y-Axis Acc

KeyboardInterrupt: 